
<br>
## CS 429: Information Retrieval
## Lecture 2: Indexing

<br>

### Dr. Aron Culotta
### Illinois Institute of Technology 

<br><br><br>
<br><br><br>

---



# Indexing Pipeline

1. Collect documents
2. Tokenize
3. Normalize
4. Index

# Indexing Pipeline

document $\xrightarrow{tokenize}$ (tokens, types) $\xrightarrow{normalize}$ terms $\xrightarrow{index}$ inverted index

In [ ]:
# 1. Collect documents.
document = "he didn't know where he worked."

# 2. Tokenize
tokens = ["he", "didn't", "know", "where", "he", "worked"] # split; remove punctuation
types = ["he", "didn't", "know", "where", "worked"] # unique tokens.

# 3. Normalize
# remove common words like 'where'; collapse word forms like worked -> work
terms = ["he", "didnt", "know", "work"]

# 4. Index
index = {'he': [0],
         'didnt': [0],
         'know' : [0],
         'work' : [0]}

**e·quiv·a·lence class** (/i'kwivələns klas/) *n.*

1. A subset whose elements are equivalent according to some relation $\sim$ .

    $S' \subseteq S \hspace{.2cm}$ s.t. $\hspace{.2cm}x_i \sim x_j \hspace{.1cm} \forall x_i, x_j \in S'$
    
    E.g., consider the set \{dog, cat, spider\} and the relation "has same number of legs". Then there are two equivalence classes: \{dog, cat\} and \{spider\}.

**to·ken** (/tōkən/) *n*.

1. A sequence of characters in a document that form a meaningful unit.
2. The output of a *tokenizer*.

**type** (/tīp/) *n.*

1. An equivalence class of *tokens* under the string equality relation.
2. By analogy to OO-programming, class:object :: type:token

    *e.g., "to be or not to be"* $\xrightarrow{tokenize}$ *tokens={to, be, or, not, to, be}* $\hspace{.2cm}$  *types={to, be, or, not}*
    
    The type 'to' is an equivalence class containing the first and fifth tokens of the document.


**term** (/tərm/) *n.*

1. An equivalence class of *types* under the relation "have the same normalized form."
2. The keys in the inverted index.

    e.g., types={John, john, aren't, arent} $\xrightarrow{normalize}$ terms={john, arent}.

    The term "john" is an equivalence class containing types {John, john}.
    
    The term "arent" is an equivalence class containing types {arent, aren't}.
    
    
<br><br><br>
    

# Tokenization

**to·ken·i·za·tion** (/ˈtōkən izā-shən/) *n.*

1. The process of splitting a document into tokens.

    *Simplest approach*: split on whitespace.


In [ ]:
document.split()

In [ ]:
'what     about multiple     spaces?'.split()

# Tokenization: Compound Nouns

- *San Francisco*;  *New York University* vs *York University*
- Solved somewhat by *phrase indexing* (next class)

# Tokenization: Segmentation

- *Lebensversicherungsgesellschaftsangestellter*
  - "life insurance company employee"
- 我不能说中国话
- *\#androidgames*


- Statistical classification algorithms can be used to split (Part III of course). 
- Simpler: index character subsequences (*n-grams*).
  - E.g., *\#androidgames* $\rightarrow$ {*#andr, andro, ndroi, droid, roidg, ..., games*}

# Tokenization: Punctuation

- Remove all punctuation? 

  - "didn't", "www.google.com"
  - [CAR](https://www.google.com/search?q=CAR) vs [C.A.R](https://www.google.com/search?q=C.A.R.).
  - [O'Neill vs ONeill vs O Neill](https://www.google.com/#q=oneill+-o'neill&safe=active)

# Tokenization: Regular Expressions

symbol | meaning
------ | -------
\b	| Word boundary (zero width)
\d	| Any decimal digit (equivalent to [0-9])
\D	| Any non-digit character (equivalent to [^0-9])
\s	| Any whitespace character (equivalent to [ \t\n\r\f\v]
\S	| Any non-whitespace character (equivalent to [^ \t\n\r\f\v])
\w	| Any alphanumeric character (equivalent to [a-zA-Z0-9_])
\W	| Any non-alphanumeric character (equivalent to [^a-zA-Z0-9_])
\t	| The tab character
\n	| The newline character

(source: <http://nltk.org/book/ch03.html>)

In [ ]:
import re  # Regular expression module
re.split('x', 'axbxc')

In [ ]:
help(re.split)

In [ ]:
help(re)

In [ ]:
re.split('x+', 'axxxxbxxxxc')

In [ ]:
re.split('x', 'axxxxbxxxxc')

In [ ]:
re.split('\+\+', 'hi+++there')

In [ ]:
re.split('([\W\s]|t)', "what's up?")

In [ ]:
text = "A first-class ticket to the U.S.A. isn't expensive?"
re.split(' ', text)

How to remove punctuation?

In [ ]:
re.split('\W+', text)           # \W=not a word character; +=1 or more

In [ ]:
re.findall('\w+', text)         # \w=a word character [a-zA-Z0-9_]

In [ ]:
# group punctuation with following letters
re.findall('\w+|\S\w*', text)  # \S=not a space; |=OR

How to keep hyphenated words and contractions together?

In [ ]:
re.findall("\w+(?:[-']\w+)*|[-.(]+|\S\w*", text)
# (?: specifies what to match, not what to capture

In [ ]:
re.findall("(?:[A-Z]\.)+|\w+(?:[-']\w+)*|[-.(]+|\S\w*", text)

# Normalization

**nor·mal·iz·a·tion** (/ˈnôrməˌlizā-shən/) *n.*

1. The process of clustering *types* into *terms*.

    Issues include: removing common words, special characters, casing, morphology
    
<br><br><br>

# Normalization: Stop words

- Exclude common words
  - *the*, *a*, *be*
- Why?

<br><br><br>
<br><br><br>

  - save space (length of postings list is huge!)
  - no semantic content (?!)
  
  <br><br><br>
  <br><br><br>

*"[to be or not to be](https://www.google.com/search?q=to+be+or+not+to+be&oq=to+be+or+not+to+be)"* is all stop words!

# Accents/Diacritics

- naive vs. naïve
- pena (sorrow) vs peña (cliff)

- What will users enter?

# Case

- Typically, just convert everything to lowercase.
- E.g., search Google for [CAT -cat](https://www.google.com/search?q=CAT+-cat).

# Stemming / Lemmatizing

**mor·phol·o·gy** (/môrˈfäləjē/) *n.*

1. (*Linguistics*) The study of the rules governing how words may take different forms in a language.

*E.g.* 

- Pluralization: *dog* $\xrightarrow{pluralize}$ *dogs* ; *goose* $\xrightarrow{pluralize}$ *geese*


- Tense: *play* $\xrightarrow{past.tense}$ *played* ; *go* $\xrightarrow{past.tense}$ *went*

**stem** (/stem/) *v.*

1. To normalize based on crude morphology heuristics.

    *E.g. remove all "-s" and "-ed" suffixes*

**lem·ma·tize** (ˈleməˌtīz/) *v.*

1. To create equivalence classes of word types using the morphological rules of a language.

    *Often relies on **part-of-speech** tagging to select rules*.

    *E.g. if * bed * is a noun, then do not remove * -ed *suffix.*

# Simple stemmer



In [ ]:
s = 'hello'
s[:10]

In [ ]:
def stem(word):
    for suffix in ['ies', 's', 'ed', 'ing']: # order matters!
        if word.endswith(suffix):
            return word[:-len(suffix)]

**What can go wrong?**

# Stemming Errors

- **over-stemming**: merge types that should not be merged.
- **under-stemming**: fail to merge types that should be merged.

In [ ]:
types = ['tied', 'ties', 'tis', 'bed', 'cities']
[stem(w) for w in types]

**How does this affect search?**

# Porter Stemmer

- Very commonly used stemmer with a complex set of heuristics.

In [ ]:
from nltk.stem import PorterStemmer # See nltk.org (`pip install nltk`)
# http://tartarus.org/~martin/PorterStemmer/
# Original paper: http://web.simmons.edu/~benoit/lis466/PorterStemmingAlgorithm.pdf
porter = PorterStemmer()
print(types)
[porter.stem(x) for x in types]

In [ ]:
porter.stem('city')

In [ ]:
types = ['bed', 'kiss',
         'tied', 'tis',
         'universal', 'university',
         'experiment', 'experience',
         'past', 'paste',
         'alumnus', 'alumni',
         'adhere', 'adhesion',
         'create', 'creation']
porter_results = [porter.stem(x) for x in types]
porter_results

# WordNet Lemmatizer

In [ ]:
from nltk.stem.wordnet import WordNetLemmatizer
# See description: https://wordnet.princeton.edu/wordnet/man/morphy.7WN.html
lemm = WordNetLemmatizer()
lemm_results = [lemm.lemmatize(x) for x in types]
print('%15s\t%15s\t%15s' % ('type', 'porter', 'lemmatizer'))
print('\n')
print('\n'.join(['%15s\t%15s\t%15s' % (t[0], t[1], t[2])
                 for t in zip(types, porter_results, lemm_results)]))

In [ ]:
print(lemm.lemmatize('are'))
print(lemm.lemmatize('is'))

In [ ]:
print(lemm.lemmatize('are', 'v'))
print(lemm.lemmatize('is', 'v'))

# A principled approach?

Given the many number of ways to preprocess text, how do we know which one is best?

Approaches:

- Assume types that appear in similar contexts can be merged.
  - e.g., *universally* and *universal* appear in similar documents, but not *university*.

- Learn from user behavior
  - e.g., users click on very different search results if they search for *universal* vs *university*.
  
We'll explore both later in the course.